In [52]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
from Bio.SVDSuperimposer import SVDSuperimposer
import nglview as nv

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First we load the pdbs as is

In [53]:
loc = '../../pymdna/atomic/'
base_G = md.load(loc+'BDNA_G.pdb')

loc = '/Users/thor/surfdrive/Projects/pymdna/examples/non_canonical/'
base_TC1 = md.load(loc+'BDNA_H-draft.pdb')
                
base_G, base_TC1

OSError: No such file: /Users/thor/surfdrive/Projects/pymdna/examples/non_canonical/BDNA_H-draft.pdb

Compare 2AP with A nuceleobase, find atoms that are a good fit (aka fit planar nucleobases on top of each other)
`['N9', 'N3', 'C6', 'C4', 'C2', 'C5', 'N7']`

In [54]:
merge = base_G.stack(base_TC1)
view = nv.show_mdtraj(merge)
view

NGLWidget()

Now we use these atoms to move the non-canonical base to the same plane as the standard base A and compare the x,y,z coorindates of the atoms that are used for the ReferenceBase to construct the basis of the nucleobase. The fitted structure we can now add to the atomic directory ('BDNA_H.pdb'), and edit the `base_pair_map` `dict` in `generator.py` to encode the new base.

In [38]:
 
def get_rot_and_trans(subtraj_A,subtraj_B):
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
        even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = np.dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def fit_B_on_A(A, B, selection_A, selection_B):
    
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(selection_A)
    subtraj_B = B.atom_slice(selection_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)
    
    # do the superimposition of B on A and subsitute old with new xyz of B
    sup_B = apply_superimposition(B, rot, tran)

    # remove overlapping backbone atoms from B 
    selection_to_keep = [at.index for at in  B.top.atoms if at.index not in selection_B]

    new_B = sup_B
    return new_B, rms

def get_fit_indices(traj):
    residue = traj.top.residue(0)
    #print(residue, [at.name for at in residue._atoms])
    # res_indices = [at.index  for at in residue.atoms if at.name in ["P",'OP1','OP2',"C4'","C3'","C5'","O4'","O3'","C1\'"]]
    res_indices = [at.index  for at in residue.atoms if at.name in ["P","C5'","C1\'"]]
    #res_indices = [at.index  for at in residue.atoms if at.name in ['N1','N9', 'N3', 'C6', 'C4', 'C2', 'C5', 'N7']]

    print(res_indices)
    return res_indices
            
def fit_bases(base_A, base_B):
    """ fit bases on trajectory """

    selection_A = get_fit_indices(base_A)
    selection_B = get_fit_indices(base_B)

    # Fits B on A
    return fit_B_on_A(A=base_A, B=base_B, selection_A=selection_A, selection_B=selection_B)


new_base_TC1, rms = fit_bases(base_G, base_TC1)
print(rms)
# Uncomment if you don't want hydrogens
#new_base_2AP.atom_slice(new_base_2AP.top.select('not element type hydrogen'))#.save('BDNA_P.pdb')
new_base_TC1.save_pdb('BDNA_H.pdb')


res_A = mdna.ReferenceBase(base_G)
res_ref = mdna.ReferenceBase(new_base_TC1)

print(res_A.base_type, res_ref.base_type)
print(res_A.C1_coords, res_ref.C1_coords)
print(res_A.N_coords, res_ref.N_coords)
print(res_A.C_coords, res_ref.C_coords)


[0, 4, 10]
[0, 4, 8]
0.021636762694314603
purine pyrimidine
[[-0.2477  0.5399 -0.    ]] [[-0.26305276  0.5237298   0.0034253 ]]
[[-0.1289  0.4551 -0.    ]] [[-0.14486474  0.43671823  0.00651786]]
[[-0.1265  0.3177 -0.    ]] [[-0.16792381  0.29508835  0.00297737]]


In [46]:
# Make control points
control_points = mdna.Shapes.line(length=3,num_points=100)

# Generate spline frames from control points
spline = mdna.SplineFrames(
    control_points=control_points, 
    frame_spacing=0.34,
    frame_tolerance=0.05,
    degree=3, 
    num_points = 10000,
    closed=False,
    verbose=False)

spline.twist_frames()
spline.frames.shape
sequence = 'M'*spline.frames.shape[0]
dna = mdna.StructureGenerator(spline=spline,sequence=sequence)
traj = dna.traj
view = nv.show_mdtraj(traj)
view.clear()
view.add_representation('ball+stick', selection='all')  
view



/Users/thor/surfdrive/Projects/pymdna/pymdna/utils.py:54: RuntimeWarning: invalid value encountered in divide
  omega_normalized = np.where(omega_norm != 0, omega / omega_norm, omega)


NGLWidget()